In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import flood_fill

In [ ]:
def preprocess(fname):
    garden = []
    with open(fname, "r") as f:
        for line in f.readlines():
            line_conv = [ord(x) for x in line.strip()]
            garden.append(line_conv)
    matrix = np.array(garden)
    return matrix

# nearest neighbors
nn = [( 0,  1),
      ( 1,  0),
      (-1,  0),
      ( 0, -1)]

def get_num_boundaries(segment, idx):
    pad_width = 1
    segment = np.pad(segment, pad_width=pad_width)
    boundaries = 0
    for n in nn:
        rn, cn = idx[0] + n[0], idx[1] + n[1]
        if not segment[rn + pad_width, cn + pad_width]:
            boundaries += 1
    return boundaries

def get_perimeter(segment):
    peri = 0
    segment_idx = list(zip(*(segment==True).nonzero()))
    for idx in segment_idx:
        peri += get_num_boundaries(segment, idx)
    return peri

def part1(garden):
    filled = 0 * garden
    remaining_idx = [(0,0)]
    cost = 0

    while remaining_idx:
        seed_point = remaining_idx[0]
        segment = flood_fill(garden, seed_point=seed_point, new_value=0, connectivity=1)==0
        area = np.sum(segment)
        peri = get_perimeter(segment)            
        filled[segment] = 1
        cost += area * peri
        remaining_idx = list(zip(*(filled==0).nonzero()))

    return cost

garden = preprocess("day12_example.txt")
part1_example_sol = part1(garden)
print(f"Part 1 solution for example data: {(part1_example_sol)}")
assert (part1_example_sol) == 1930

In [ ]:
garden = preprocess("day12_example2.txt")
part1_example2_sol = part1(garden)
print(f"Part 1 solution for 2nd example data: {(part1_example2_sol)}")
assert (part1_example2_sol) == 772

In [ ]:
garden = preprocess("day12_input.txt")
part1_sol = part1(garden)
print(f"Part 1 solution: {part1_sol}")

In [5]:
def get_num_segments(img):
    filled = 0 * img
    remaining_idx = [(0,0)]
    counter = 0

    while remaining_idx:
        seed_point = remaining_idx[0]
        segment = flood_fill(img, seed_point=seed_point, new_value=0, connectivity=1)==0
        filled[segment] = 1
        counter += 1
        remaining_idx = list(zip(*(filled==0).nonzero()))

    return counter

In [ ]:
def part2(garden):
    filled = 0 * garden
    remaining_idx = [(0,0)]
    cost = 0

    while remaining_idx:
        seed_point = remaining_idx[0]
        segment = flood_fill(garden, seed_point=seed_point, new_value=0, connectivity=1)==0
        padded_segment = np.pad(segment, pad_width=1).repeat(2, axis=0).repeat(2, axis=1)
        diffx = (np.diff(padded_segment, axis=0, prepend=0))
        diffy = (np.diff(padded_segment, axis=1, prepend=0))
        area = np.sum(segment)
        num_edges = get_num_segments(diffx) - 1 + get_num_segments(diffy) - 1
        filled[segment] = 1
        cost += area * num_edges
        remaining_idx = list(zip(*(filled==0).nonzero()))

    return cost

garden = preprocess("day12_example.txt")
part2_example_sol = part2(garden)
print(f"Part 2 solution for example data: {(part2_example_sol)}")
assert (part2_example_sol) == 1206

In [ ]:
garden = preprocess("day12_input.txt")
part2_sol = part2(garden)
print(f"Part 2 solution: {part2_sol}")

In [8]:
def visu(segment):
    fig, axs = plt.subplots(ncols=4, figsize=(12,12))
    padded_segment = np.pad(segment, pad_width=1).repeat(2, axis=0).repeat(2, axis=1)
    diffx = (np.diff(padded_segment, axis=0, prepend=0))
    diffy = (np.diff(padded_segment, axis=1, prepend=0))

    axs[0].pcolormesh(padded_segment, edgecolors='w', linewidth=0.5)
    axs[1].pcolormesh(diffx, edgecolors='w', linewidth=0.5)
    axs[2].pcolormesh(diffy, edgecolors='w', linewidth=0.5)
    axs[3].pcolormesh(np.abs(diffx) | np.abs(diffy), edgecolors='w', linewidth=0.5)

    for ax in axs:
        ax.set_aspect('equal')

    plt.show()